## Library imports

In [1]:
import cv2
import pandas as pd
from sort.sort import *
from classify_by_movement import *
from calculate_features import *

In [2]:
# Show graph inline
%matplotlib inline

## Show results of video

In [10]:
# Open the video file
video_number = str(11)
fps = 50  # Replace with your video's frame rate

#video_path = 'data/data_orig/val/'+video_number+'/'+video_number+'.mp4'
video_path = '../data/ViSEM_Tracking_extended/train/'+ video_number + '/' + video_number + '.mp4'

## Calculate centroid

In [5]:
# Load the tracking data from a CSV file
df = pd.read_csv('../results/sperm_tracking_individual/sperm_tracking_data_' + video_number + '.csv')

# Calculate velocity for each track_id
df['velocity_x'] = 0.0
df['velocity_y'] = 0.0
df['speed'] = 0.0

# Frame rate of the video (frames per second)
dt = 1 / fps  # Time interval between frames

# Group by track_id and calculate velocity
for track_id, group in df.groupby('track_id'):
    # Calculate displacement (delta x and delta y)
    group['delta_x'] = group['cx'].diff()
    group['delta_y'] = group['cy'].diff()

    # Calculate velocity (pixels per second)
    group['velocity_x'] = group['delta_x'] / dt
    group['velocity_y'] = group['delta_y'] / dt

    # Calculate speed (magnitude of velocity)
    group['speed'] = (group['velocity_x']**2 + group['velocity_y']**2)**0.5
    
    # Calculate mean and maximum velocity
    group["mean_velocity"] = group['speed'].mean()
    group["max_velocity"] = group['speed'].max()
    
    df.loc[group.index, ['mean_velocity', 'max_velocity']] = group[['mean_velocity', 'max_velocity']].fillna(0)
    
    # Update the original DataFrame
    df.loc[group.index, ['velocity_x', 'velocity_y', 'speed']] = group[['velocity_x', 'velocity_y', 'speed']].fillna(0)

# Save the updated DataFrame with velocity data
df.to_csv('../results/data_sperm_track_and_velocity/sperm_tracking_with_velocity_' + video_number + '.csv', index=False)

## Show tracking and velocitty in video

In [12]:
# Load the tracking data with velocity
df = pd.read_csv('../results/data_sperm_track_and_velocity/sperm_tracking_with_velocity_' + video_number + '.csv')
df_class = pd.read_csv('../results/data_features_labelling/dataset_4c_11.csv')

#df_class = df_class[df_class['sperm_id'].isin([35])]

# Open the video file
cap = cv2.VideoCapture(video_path)

# Set the slow-motion factor (1 is for normal velocity and ~0 for slow velocity)
slow_motion_factor = 1 # Adjust this value as needed
original_delay = int(1000 / fps)  # Delay in milliseconds
new_delay = int(original_delay / slow_motion_factor) # Calculate the new delay between frames

# Variables
trajectories = {}
total_class = 4

# Process the video frame by frame
frame_id = 0

    
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the data for the current frame
    frame_data = df[df['frame_id'] == frame_id]

    # Draw sperm items
    for _, row in frame_data.iterrows():
        cx, cy = int(row['cx']), int(row['cy'])
        
        # Set sperm trajectory
        track_id = int(row['track_id'])
        if track_id not in trajectories:
            trajectories[track_id] = []
        trajectories[track_id].append((cx, cy))
        
        # Set class
        n_class = 0
        try:
            n_class = int(df_class[df_class['sperm_id']==track_id]['label'])
        except:
            continue
        
        
        # Define the color according to the class. Estructure: 2c/3c/4c
        color = (0, 0, 0)
        if n_class == 0:
            color = (0, 255, 0) # Green - Progressive/Progressive/Rapdly progressive
        elif n_class == 1:
            color = (255, 0, 0) # Blue - Non progressive/Non progressive/Slowly progressive
        elif n_class == 2:
            if total_class == 3:
                color =  (0, 0, 255)  # Red - -/-/Inmotile
            else:
                color = (0, 255, 255) # Yellow - -/Inmotile/Non progressive
        elif n_class == 3:
            color =  (0, 0, 255)  # Red - -/-/Inmotile
    
        # Draw path
        for i in range(1, len(trajectories[track_id])):
            cv2.line(frame, (int(trajectories[track_id][i - 1][0]),int(trajectories[track_id][i - 1][1])), (int(trajectories[track_id][i][0]),int(trajectories[track_id][i][1])), color, 1)

        # Draw rectangle for sperm head
        cv2.rectangle(frame, (int(row['xmin']), int(row['ymin'])), (int(row['xmax']), int(row['ymax'])), color, 1)
        
        # Draw id of sperm
        cv2.putText(frame, str(track_id), (cx + 10, cy), cv2.FONT_HERSHEY_PLAIN, 1.2, (255, 255, 255), 1,  cv2.LINE_AA )


    # Display the frame
    cv2.imshow('Sperm Velocity', frame)

    # Save the frame (optional)
    #cv2.imwrite(f'output/frame_{frame_id:04d}.jpg', frame)

    # Wait for the calculated delay
    if cv2.waitKey(new_delay) & 0xFF == ord('q'):
        break
    
    # Increase frame
    frame_id += 1

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()